# Revista de Contabilidade da UFBA

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(query, url_search, pg_num=1):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    div_results = soup.find_all('h2', {'class': 'title'})
    a_links = []
    for item in div_results:
        try:
            a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(query, url_search, pg_num)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://periodicos.ufba.br/index.php/rcontabilidade/search/search?query=__query__&searchJournal=21&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'

listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)


['https://periodicos.ufba.br/index.php/rcontabilidade/article/view/35190',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/42196',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/23010',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/23259',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/26456',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/8357',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/30783',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/22946',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/23868',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/16635',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/23867',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/23817',
 'https://periodicos.ufba.br/index.php/rcontabilidade/article/view/23864',
 'https://periodicos.ufba.

In [6]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [7]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [8]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date}

In [9]:
def captura_resumo(soup):
    try:
        abstract = soup.find('section', {'class': 'item abstract'}).text
        abstract = abstract.replace('Resumo', '')
        abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [10]:
def captura_palavras_chave(soup):
    try:
        kw_tag = soup.find('section', {'class': 'item keywords'})
        kw = kw_tag.find('span', {'class': 'value'}).text.replace(',', '.').replace(';', '.')
        kw = ' '.join(kw.split())
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [11]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [12]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [13]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista de Contabilidade da UFBA'}
    url_search_text = 'https://periodicos.ufba.br/index.php/rcontabilidade/search/search?query=__query__&searchJournal=21&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y-%m-%d')
    return df

## Pesquisa por artigos

In [14]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2020-09-05,ANÁLISE DO DISCLOSURE AMBIENTAL DAS EMPRESAS L...,"O presente estudo objetivou analisar, consider...",Evidenciação Ambiental. Índice de Carbono Efic...,4,Alany Taysy Fernandes de Lima,Renato Henrique Gurgel Mota,Alexsandro Gonçalves da Silva Prado,Marlla Feitosa de Oliveira,,,Universidade Federal Rural do Semi-árido,Universidade Federal do Rio Grande do Norte (U...,UFERSA,Universidade do Vale do Rio dos Sinos (Unisinos),,,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA
1,2021-07-04,Existe um gap entre o conteúdo formativo da di...,O estudo objetivou analisar a existência de um...,Controladoria. Controller. Formação acadêmica....,4,Ellen Carine Mello dos Santos,Marciano da Silveira Piazentini,Gabriel Bonetto Bampi,Fernando Maciel Ramos,,,Universidade do Contestado,Universidade do Contestado,Universidade do Contestado,Universidade do Contestado,,,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA
2,2018-12-28,Informações da Contabilidade Ambiental sobre C...,Este trabalho objetivou em elaborar o Balanço ...,Recurso Hídrico. Contabilidade Ambiental. Regi...,1,Jozenei Silva Pereira,,,,,,Faculdade Santo Antonio / Prefeitura Municipal...,,,,,,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA


In [15]:
df.to_csv('../data/RCUFBA.csv', index=False)

In [16]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2020-09-05,ANÁLISE DO DISCLOSURE AMBIENTAL DAS EMPRESAS L...,"O presente estudo objetivou analisar, consider...",Evidenciação Ambiental. Índice de Carbono Efic...,4,Alany Taysy Fernandes de Lima,Renato Henrique Gurgel Mota,Alexsandro Gonçalves da Silva Prado,Marlla Feitosa de Oliveira,,,Universidade Federal Rural do Semi-árido,Universidade Federal do Rio Grande do Norte (U...,UFERSA,Universidade do Vale do Rio dos Sinos (Unisinos),,,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA
1,2021-07-04,Existe um gap entre o conteúdo formativo da di...,O estudo objetivou analisar a existência de um...,Controladoria. Controller. Formação acadêmica....,4,Ellen Carine Mello dos Santos,Marciano da Silveira Piazentini,Gabriel Bonetto Bampi,Fernando Maciel Ramos,,,Universidade do Contestado,Universidade do Contestado,Universidade do Contestado,Universidade do Contestado,,,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA
2,2018-12-28,Informações da Contabilidade Ambiental sobre C...,Este trabalho objetivou em elaborar o Balanço ...,Recurso Hídrico. Contabilidade Ambiental. Regi...,1,Jozenei Silva Pereira,,,,,,Faculdade Santo Antonio / Prefeitura Municipal...,,,,,,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA
3,2017-12-07,A INFLUÊNCIA DOS FATORES CONTINGENCIAIS E NÃO ...,Questões socialmente responsáveis transcendem ...,Dimensão econômica e ambiental. Ecoeficiência....,5,Diones Fernandes Zanin,Luiz Panhoca,Lauro Brito de Almeida,Frederico Fonseca da Silva,Marcielle Anzilago,,,,,,,,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA
4,2019-02-08,SUSTENTABILIDADE AMBIENTAL: UM ESTUDO DE CASO ...,Este estudo tem como objetivo analisar o índic...,Sustentabilidade. CeasaMinas. SICOGEA. Contabi...,2,Karina Batista de Aquino,Tatiane de Oliveira Marques,,,,,Universidade Federal de Minas Gerais – UFMG,Universidade Federal do Paraná - UFPR,,,,,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA
5,2015-01-08,CONTABILIDADE E SUSTENTABILIDADE: UMA ANÁLISE ...,O objetivo do presente estudo foi verificar ju...,Contabilidade. Sustentabilidade. Periódicos Ci...,2,Luciano Gomes dos Reis,Marcelo Resquetti Tarifa,,,,,Universidade Estadual de Londrina,Universidade Estadual de Londrina,,,,,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA
6,2019-12-31,Auditoria dos Relatórios de Sustentabilidade: ...,Este estudo buscou verificar a relação entre o...,Auditoria. Relatórios de Sustentabilidade. Teo...,1,Taianan Alves Uzeda Luna,,,,,,Faculdade Adventista da Bahia,,,,,,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA
7,2018-12-28,Qualidade Informacional versus Nível de Govern...,O objetivo do estudo foi analisar a qualidade ...,Governança Coorporativa. Qualidade das Informa...,6,Adriana Maria Miguel Peixe,Jessika Nassif Korontai,Neusa Higa,Denise Kowalski,Ademir Clemente,Blênio Cezar Severo Peixe,UNIVERSIDADE FEDERAL DO PARANÁ - UFPR,Universidade Federal do Paraná - UFPR,Universidade Federal do Paraná - UFPR,Universidade Federal do Paraná - UFPR,Universidade Federal do Paraná - UFPR,Universidade Federal do Paraná - UFPR,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA
8,2017-12-07,DIVULGAÇÃO DE INFORMAÇÕES SOCIAIS E AMBIENTAIS...,"O estudo analisa, sob a ótica da Teoria da Leg...",Evidenciação. Responsabilidade social. Legitim...,2,Francielle Rodrigues Voltarelli,Aládio Zanchet,,,,,Unioeste - Universidade Estadual do Oeste do P...,Unioeste - Universidade Estadual do Oeste do P...,,,,,https://periodicos.ufba.br/index.php/rcontabil...,Revista de Contabilidade da UFBA
9,2017-08-28,ANÁLISE DA EVIDENCIAÇÃO DA S